# Keras/Tensorflow implementation of the CNN-LSTM model for Limit Order Book forecasting

This notebook contains an independently developed Keras/Tensorflow implementation of the CNN-LSTM model for Limit Order Book forecasting originally proposed by Zhang *et al.* (https://arxiv.org/pdf/1808.03668.pdf). The current implementation was adopted in the paper written by Briola *et al.* (https://arxiv.org/pdf/2007.07319.pdf).

The interested reder can find the original implementation of the model at https://github.com/zcakhaa/DeepLOB-Deep-Convolutional-Neural-Networks-for-Limit-Order-Books/blob/master/jupyter/run_train_represent.ipynb.

In [4]:
import tensorflow as tf
from tensorflow.keras import *
from tensorflow.keras.layers import *

In [9]:
def cnn_lstm_model():


    inputs = Input(shape=(WINDOW_SIZE, NUMBER_FEATURES, 1,))
    first_layer = Conv2D(filters=16, kernel_size=(1, 2), strides=(1, 2))(inputs)
    first_leaky_relu_layer = LeakyReLU(alpha=0.05)(first_layer)
    first_int_layer = Conv2D(filters=16, kernel_size=(4, 1), padding='same')(first_leaky_relu_layer)
    second_int_layer = Conv2D(filters=16, kernel_size=(4, 1), padding='same')(first_int_layer)
    second_layer = Conv2D(filters=16, kernel_size=(1, 2), strides=(1, 2))(second_int_layer)
    second_leaky_relu_layer = LeakyReLU(alpha=0.05)(second_layer)
    third_int_layer = Conv2D(filters=16, kernel_size=(4, 1), padding='same')(second_leaky_relu_layer)
    fourth_int_layer = Conv2D(filters=16, kernel_size=(4, 1), padding='same')(third_int_layer)
    third_layer = Conv2D(filters=16, kernel_size=(1, 10))(fourth_int_layer)
    third_leaky_relu_layer = LeakyReLU(alpha=0.05)(third_layer)
    fifth_int_layer = Conv2D(filters=16, kernel_size=(4, 1), padding='same')(third_leaky_relu_layer)
    sixth_int_layer = Conv2D(filters=16, kernel_size=(4, 1), padding='same')(fifth_int_layer)

    #### Inception module ####

    tower_1 = Conv2D(32, (1,1), padding='same')(sixth_int_layer)
    tower_1_first_leaky = LeakyReLU(alpha=0.05)(tower_1)
    tower_1 = Conv2D(32, (3,1), padding='same')(tower_1_first_leaky)
    tower_1_second_leaky = LeakyReLU(alpha=0.05)(tower_1)

    tower_2 = Conv2D(32, (1,1), padding='same')(sixth_int_layer)
    tower_2_first_leaky = LeakyReLU(alpha=0.05)(tower_2)
    tower_2 = Conv2D(32, (5,1), padding='same', activation='relu')(tower_2_first_leaky)
    tower_2_second_leaky = LeakyReLU(alpha=0.05)(tower_2)

    tower_3 = MaxPooling2D((3,1), strides=(1,1), padding='same')(sixth_int_layer)
    tower_3 = Conv2D(32, (1,1), padding='same')(tower_3)
    tower_3_second_leaky = LeakyReLU(alpha=0.05)(tower_3)
    
    ### CONCATENATION ##
    output_inception = layers.concatenate([tower_1_second_leaky, 
                                           tower_2_second_leaky, 
                                           tower_3_second_leaky], axis = 3)

    newdim = tuple([x for x in output_inception.shape.as_list() if x != 1 and x is not None])
    reshape_layer = Reshape(newdim) (output_inception)
    
    ### LSTM LAYER ###

    lstm_layer = LSTM(64)(reshape_layer)

    final_layer = Dense(3, activation='softmax')(lstm_layer)
    model = Model(inputs, final_layer)
    
    return model